In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
odds_filter=1.75
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
time_now_formatted = time_now.strftime("%Y-%m-%d")
data=pd.read_sql_query("Select distinct * from results_hard_1 --where date > '2024-01-01'",con=devengine)
data["Fav_Odds"] = data.Fav_Odds.astype(float)
data["Dog_Odds"] = data.Dog_Odds.astype(float)
data['Fav_Win'] = data['Winner']==data['Fav']
data=data[data['Fav_Odds']<=odds_filter]

In [ ]:
test_data = data.copy()
test_data1 = data.copy()
test_data['Winner_Dog'] = test_data['Dog'] == test_data['Winner']
test_data['Winner_EloDog'] = test_data['Elo_Fav'] != test_data['Winner']
column_order = ['Winner_Dog', 'Winner_EloDog'] + [col for col in test_data if col not in ['Winner_Dog', 'Winner_EloDog']]
test_data = test_data[column_order]
test_data=test_data.drop(columns=['Winner_Odds'])
test_data=test_data[[ 'fav_percent', 'dog_percent','fav_rank', 'dog_rank', 'Elo_Fav_Elo', 'Elo_Dog_Elo', 'Fav_Top100', 'Dog_Top100','Winner','Fav','Dog','Fav_Odds','Dog_Odds','Elo_Fav']]

# Create a single set of random indices for all columns
random_order = np.random.permutation(test_data.index)

# Create new columns with randomized order for specific columns
test_data['Player_1'] = test_data.apply(lambda row: np.random.choice([row['Fav'], row['Dog']]), axis=1)
test_data['Player_2'] = test_data.apply(lambda x: x['Fav'] if x['Player_1']==x['Dog'] else x['Dog'],axis=1)
test_data['Player_1_Odds'] = test_data.apply(lambda x: x['Fav_Odds'] if x['Player_1']!=x['Dog'] else x['Dog_Odds'],axis=1)
test_data['Player_2_Odds'] = test_data.apply(lambda x: x['Fav_Odds'] if x['Player_1']==x['Dog'] else x['Dog_Odds'],axis=1)
test_data['Player_1_Rank'] = test_data.apply(lambda x: x['fav_rank'] if x['Player_1']!=x['Dog'] else x['dog_rank'],axis=1)
test_data['Player_2_Rank'] = test_data.apply(lambda x: x['fav_rank'] if x['Player_1']==x['Dog'] else x['dog_rank'],axis=1)
test_data['Player_1_Top100'] = test_data.apply(lambda x: x['Fav_Top100'] if x['Player_1']!=x['Dog'] else x['Dog_Top100'],axis=1)
test_data['Player_2_Top100'] = test_data.apply(lambda x: x['Fav_Top100'] if x['Player_1']==x['Dog'] else x['Dog_Top100'],axis=1)
test_data['Player_1_percent'] = test_data.apply(lambda x: x['fav_percent'] if x['Player_1']!=x['Dog'] else x['dog_percent'],axis=1)
test_data['Player_2_percent'] = test_data.apply(lambda x: x['fav_percent'] if x['Player_1']==x['Dog'] else x['dog_percent'],axis=1)
test_data['FavEloFav']=test_data['Fav']==test_data['Elo_Fav']
test_data['Fav_Elo']=test_data.apply(lambda x: x['Elo_Fav_Elo'] if (x['FavEloFav']==True) else x['Elo_Dog_Elo'],axis=1)
test_data['Dog_Elo']=test_data.apply(lambda x: x['Elo_Fav_Elo'] if (x['FavEloFav']==False) else x['Elo_Dog_Elo'],axis=1)

test_data['Player_1_Elo'] = test_data.apply(lambda x: x['Fav_Elo'] if x['Player_1']!=x['Dog'] else x['Dog_Elo'],axis=1)
test_data['Player_2_Elo'] = test_data.apply(lambda x: x['Fav_Elo'] if x['Player_1']==x['Dog'] else x['Dog_Elo'],axis=1)
test_data['Player_1_Win'] = test_data['Player_1']==test_data['Winner']
#test_data['Player_2_Elo'] = test_data.apply(lambda x: x['Elo_Dog_Elo'] if ((x['Player_1']==x['Dog'])&(x['FavEloFav2']==True)) else x['Elo_Fav_Elo'],axis=1)
# Filter columns with the word 'Player'
player_columns = [col for col in test_data.columns if 'Player' in col]

# Keep only the columns with the word 'Player'
test_data = test_data[player_columns]
test_data=test_data.drop(columns=['Player_1','Player_2','Player_1_Odds','Player_2_Odds'])

In [ ]:
import numpy as np

# Assuming your DataFrame is named 'test_data'
data['fav_percent_adj'] = np.ceil(data['fav_percent'] * 10) / 10
data['dog_percent_adj'] = np.ceil(data['dog_percent'] * 10) / 10



In [ ]:
# Assuming your DataFrame is named 'test_data'
result = data.groupby(['fav_percent_adj', 'dog_percent_adj'])['Fav_Win'].agg(['mean', 'count']).reset_index()

result.rename(columns={'mean': 'Fav_Win_Percentage', 'count': 'Match_Count'}, inplace=True)


result.sort_values(by='Fav_Win_Percentage').to_pickle('Fav_Win_Percentage')

In [ ]:
hard_today=data
Fav_Win_Percentage=result
hard_today['fav_percent_adj'] = np.ceil(hard_today['fav_percent'] * 10) / 10
hard_today['dog_percent_adj'] = np.ceil(hard_today['dog_percent'] * 10) / 10
hard_today_win_percent=pd.merge(hard_today,Fav_Win_Percentage,left_on=['fav_percent_adj','dog_percent_adj'],right_on=['fav_percent_adj','dog_percent_adj'])
#hard_today_win_percent=hard_today_win_percent[hard_today_win_percent['Resulted']!='Falxse']
hard_today_win_percent=hard_today_win_percent[hard_today_win_percent['Sex']=='Womens']
results=hard_today_win_percent[['Fav_Win_Percentage','Fav','Fav_Odds','fav_percent','dog_percent','Dog','Dog_Odds','Fav_Win']]#.sort_values(by=['Time'],ascending=True )

results=results[results['Fav_Win_Percentage']>0.79]
len(results[results['Fav_Win']==True])/len(results),len(results)
#results

(1.0, 34)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Assuming your data is stored in a DataFrame named 'data'
# You may need to adjust column names accordingly

# Extract features (X) and target variables (y)
X = test_data.drop(columns='Player_1_Win')
y_dog = test_data['Player_1_Win']

# Convert categorical variable 'Sex' to numerical using one-hot encoding
#X = pd.get_dummies(X, columns=['Sex'], drop_first=True)

# Split the data into training and testing sets
X_train_dog, X_test_dog, y_train_dog, y_test_dog = train_test_split(X, y_dog, test_size=0.2, random_state=42)
#X_train_elodog, X_test_elodog, y_train_elodog, y_test_elodog = train_test_split(X, y_elodog, test_size=0.2, random_state=42)

# Initialize logistic regression models
model_dog = LogisticRegression()
#model_elodog = LogisticRegression()

# Initialize logistic regression models with increased max_iter
model_dog = LogisticRegression(max_iter=100000)
#model_elodog = LogisticRegression(max_iter=100000)

# Train the models
model_dog.fit(X_train_dog, y_train_dog)
#model_elodog.fit(X_train_elodog, y_train_elodog)

# Make predictions on the test set
y_pred_dog = model_dog.predict(X_test_dog)
#y_pred_elodog = model_elodog.predict(X_test_elodog)

# Evaluate the models
accuracy_dog = accuracy_score(y_test_dog, y_pred_dog)
#accuracy_elodog = accuracy_score(y_test_elodog, y_pred_elodog)

print(f"Accuracy for Winner_Dog: {accuracy_dog}")
#print(f"Accuracy for Winner_EloDog: {accuracy_elodog}")

# Additional evaluation metrics
print("Classification Report for Winner_Dog:")
print(classification_report(y_test_dog, y_pred_dog))

#print("Classification Report for Winner_EloDog:")
#print(classification_report(y_test_elodog, y_pred_elodog))


Accuracy for Winner_Dog: 0.9230769230769231
Classification Report for Winner_Dog:
              precision    recall  f1-score   support

       False       0.86      0.86      0.86         7
        True       0.95      0.95      0.95        19

    accuracy                           0.92        26
   macro avg       0.90      0.90      0.90        26
weighted avg       0.92      0.92      0.92        26



In [ ]:
hard_today=pd.read_pickle('.\Hard_Today')
hard_today=hard_today[hard_today['Fav_Odds']>odds_filter]
#hard_today=hard_today[hard_today['Resulted']=='True']

test_data=hard_today[['Resulted', 'fav_percent', 'dog_percent','fav_rank', 'dog_rank', 'Elo_Fav_Elo', 'Elo_Dog_Elo', 'Fav_Top100', 'Dog_Top100','Fav','Dog','Fav_Odds','Dog_Odds','Elo_Fav','Time']].copy()

# Create a single set of random indices for all columns
random_order = np.random.permutation(test_data.index)

# Create new columns with randomized order for specific columns
test_data['Player_1'] = test_data.apply(lambda row: np.random.choice([row['Fav'], row['Dog']]), axis=1)
test_data['Player_2'] = test_data.apply(lambda x: x['Fav'] if x['Player_1']==x['Dog'] else x['Dog'],axis=1)
test_data['Player_1_Odds'] = test_data.apply(lambda x: x['Fav_Odds'] if x['Player_1']!=x['Dog'] else x['Dog_Odds'],axis=1)
test_data['Player_2_Odds'] = test_data.apply(lambda x: x['Fav_Odds'] if x['Player_1']==x['Dog'] else x['Dog_Odds'],axis=1)
test_data['Player_1_Rank'] = test_data.apply(lambda x: x['fav_rank'] if x['Player_1']!=x['Dog'] else x['dog_rank'],axis=1)
test_data['Player_2_Rank'] = test_data.apply(lambda x: x['fav_rank'] if x['Player_1']==x['Dog'] else x['dog_rank'],axis=1)
test_data['Player_1_Top100'] = test_data.apply(lambda x: x['Fav_Top100'] if x['Player_1']!=x['Dog'] else x['Dog_Top100'],axis=1)
test_data['Player_2_Top100'] = test_data.apply(lambda x: x['Fav_Top100'] if x['Player_1']==x['Dog'] else x['Dog_Top100'],axis=1)
test_data['Player_1_percent'] = test_data.apply(lambda x: x['fav_percent'] if x['Player_1']!=x['Dog'] else x['dog_percent'],axis=1)
test_data['Player_2_percent'] = test_data.apply(lambda x: x['fav_percent'] if x['Player_1']==x['Dog'] else x['dog_percent'],axis=1)
test_data['FavEloFav']=test_data['Fav']==test_data['Elo_Fav']
test_data['Fav_Elo']=test_data.apply(lambda x: x['Elo_Fav_Elo'] if (x['FavEloFav']==True) else x['Elo_Dog_Elo'],axis=1)
test_data['Dog_Elo']=test_data.apply(lambda x: x['Elo_Fav_Elo'] if (x['FavEloFav']==False) else x['Elo_Dog_Elo'],axis=1)

test_data['Player_1_Elo'] = test_data.apply(lambda x: x['Fav_Elo'] if x['Player_1']!=x['Dog'] else x['Dog_Elo'],axis=1)
test_data['Player_2_Elo'] = test_data.apply(lambda x: x['Fav_Elo'] if x['Player_1']==x['Dog'] else x['Dog_Elo'],axis=1)
#test_data['Player_1_Win'] = test_data['Player_1']==test_data['Winner']
#test_data['Player_2_Elo'] = test_data.apply(lambda x: x['Elo_Dog_Elo'] if ((x['Player_1']==x['Dog'])&(x['FavEloFav2']==True)) else x['Elo_Fav_Elo'],axis=1)
# Filter columns with the word 'Player'
player_columns = [col for col in test_data.columns if 'Player' in col or 'Resulted' in col or 'Time' in col]

# Keep only the columns with the word 'Player'
test_data = test_data[player_columns]
hard_today1=test_data.drop(columns=['Player_1','Player_2','Player_1_Odds','Player_2_Odds','Resulted','Time'])





# Extract features (X) from new data
X_new = hard_today1


# Convert categorical variable 'Sex' to numerical using one-hot encoding
#X_new = pd.get_dummies(X_new, columns=['Sex'], drop_first=True)

# Make predictions using the trained models
y_pred_dog_new = model_dog.predict(X_new)
#y_pred_elodog_new = model_elodog.predict(X_new)

# Make predictions using the trained models
y_pred_dog_original = model_dog.predict(X_new)
#y_pred_elodog_original = model_elodog.predict(X_new)
y_prob_dog_new = model_dog.predict_proba(X_new)[:, 1]
#y_prob_elodog_new = model_elodog.predict_proba(X_new)[:, 1]

# Add the predicted values as new columns to the original data
test_data['Pred_Player1'] = y_pred_dog_original
#hard_today['Predicted_Winner_EloDog'] = y_pred_elodog_original
test_data['Pred_Player1_Prob'] = y_prob_dog_new
test_data['Pred_Player2_Prob'] = 1-test_data['Pred_Player1_Prob']

#hard_today['Prediction_Prob_EloDog'] = y_prob_elodog_new
#hard_today['Dog_IsEloFav']=hard_today['Fav']!=hard_today['Elo_Fav']
todays_predication_hard=test_data[['Time','Player_1','Pred_Player1','Player_1_Odds','Pred_Player1_Prob','Player_2_Odds','Pred_Player2_Prob','Player_1_Rank','Player_2_Rank','Player_1_Top100','Player_2_Top100','Player_1_percent','Player_2_percent','Player_1_Elo','Player_2_Elo']][test_data['Resulted']=='False']
todays_predication_hard['Fav_Odds']=todays_predication_hard[['Player_1_Odds','Player_2_Odds']].min(axis=1)
todays_predication_hard.sort_values(by='Time',ascending=True)
#hard_today1.columns

,Time,Player_1,Pred_Player1,Player_1_Odds,Pred_Player1_Prob,Player_2_Odds,Pred_Player2_Prob,Player_1_Rank,Player_2_Rank,Player_1_Top100,Player_2_Top100,Player_1_percent,Player_2_percent,Player_1_Elo,Player_2_Elo,Fav_Odds
1,03:10,Yafan Wang,False,2.33,0.378267,1.58,0.621733,66.0,74.0,0.6,0.6,0.727273,0.571429,1809.961186,1736.208753,1.58
2,05:00,Denis Kudla,False,2.11,0.096018,1.69,0.903982,179.0,86.0,0.5,0.4,0.411765,0.444444,1720.337723,1579.435299,1.69
3,08:00,Nuno Borges,False,1.95,0.291488,1.80,0.708512,60.0,99.0,0.5,0.5,0.680000,0.517241,1776.828016,1789.356783,1.80
5,09:30,Aleksandar Vukic,True,1.54,0.612932,2.39,0.387068,69.0,104.0,0.4,0.4,0.518519,0.391304,1608.782683,1632.089508,1.54
6,11:00,Cori Gauff,True,1.36,0.648600,3.19,0.351400,3.0,9.0,0.8,0.6,0.744681,0.513514,2004.294436,1811.800432,1.36
7,22:00,Yuta Shimizu,False,2.46,0.003898,1.50,0.996102,232.0,320.0,0.2,0.0,0.666667,1.000000,1676.896338,1778.817660,1.50
8,23:30,Alexander Blockx,False,1.61,0.001450,2.23,0.998550,307.0,271.0,0.0,0.0,0.750000,0.636364,1737.630584,1655.265752,1.61
